# Supriselib and SVD on Movie Lens Dataset
![justin-lim-tloFnD-7EpI-unsplash](https://user-images.githubusercontent.com/40051540/89826144-61d26600-db1b-11ea-94f1-164fb24a5945.jpg)


In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import re
import math
import heapq
from tqdm import tqdm

from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import LeaveOneOut
from surprise.model_selection import train_test_split
from surprise import KNNBaseline
from RecommenderMetrics import RecommenderMetrics


In [3]:
movies_path = 'ml-latest-small/movies.csv'
movies_DF = pd.read_csv(movies_path) 
movies_DF.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_path = 'ml-latest-small/ratings.csv'
ratings_DF = pd.read_csv(ratings_path)
ratings_DF.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
def load_dataset():
    # Define file path
    movies_path = 'ml-latest-small/movies.csv'
    links_path = 'ml-latest-small/links.csv'
    ratings_path = 'ml-latest-small/ratings.csv'
    
    # Read file into Dataframe
    movies_DF = pd.read_csv(movies_path)    
    links_DF = pd.read_csv(links_path)     
    ratings_DF = pd.read_csv(ratings_path)
    
    ratings_DF['movieId'] = ratings_DF['movieId'].astype(str)
    ratings_DF['userId'] = ratings_DF['userId'].astype(str)
    # Reader class is used to parse a file containing ratings
    # The structure is as follows
    # user ; item ; rating ; [timestamp]
    # Dataset.load_from_file - loads a custom file
    reader = Reader(rating_scale=(1, 5))
    rating_dataset = Dataset.load_from_df(ratings_DF[['userId','movieId','rating']], reader=reader)
    
    return rating_dataset

In [7]:
def get_user_ratings(user):
    user_rating_DF = ratings_DF[ratings_DF['userId']==user][['movieId','rating']]
    user_rating_tuples = [tuple(x) for x in user_rating_DF.to_numpy()]
    return user_rating_tuples

In [11]:
def anti_test_set(test_subject,train_set):
    anti_test_set = []
    avg_rating = train_set.global_mean
    u = train_set.to_inner_uid(str(test_subject))
    
    # j is item inner id
    # user_items is a list of inner item ids that test subject u rated
    user_items = set([j for (j, _) in train_set.ur[u]])
    
    # anti-test set - (test subject, item that is not already rated by test subject, global mean)
    for i in train_set.all_items():
        if i not in user_items:
            anti_test_set.append((train_set.to_raw_uid(u),train_set.to_raw_iid(i),avg_rating))
            
    return anti_test_set

In [12]:
def get_movie_id_name():
    return movies_DF.set_index('movieId').to_dict('dict')['title']

In [28]:
def get_popularity_rank():
    popular_DF = pd.DataFrame(ratings_DF['movieId'].value_counts())
    popular_DF.columns = ['count']
    popular_DF.sort_values('count',ascending = False)
    rank = [x for x in range(len(popular_DF))]
    popular_DF['rank'] = rank
    popular_DF.drop('count',axis = 1,inplace = True)
    return popular_DF.to_dict('dict')['rank']

In [17]:
# Pick an arbitrary test subject
test_subject = 85
data = load_dataset()
movie_id_name_dict = get_movie_id_name()
algo = SVD()
print("Done...")

Done...


In [15]:
movies_of_user_loves = ratings_DF[(ratings_DF['userId'] == test_subject) & (ratings_DF['rating'] > 4.0)][['movieId','rating']]
movies_of_user_loves.sort_values(by = ['rating'],ascending = False,inplace = True)
movies_of_user_loves  = movies_of_user_loves['movieId'].tolist()

print('Movies user likes.....\n')
for movie in movies_of_user_loves:
    print(movie_id_name_dict[movie])

Movies user likes.....

Jumanji (1995)
Clear and Present Danger (1994)
Mission: Impossible (1996)
Terminator 2: Judgment Day (1991)
Jurassic Park (1993)
Fugitive, The (1993)
True Lies (1994)
Speed (1994)
Star Trek: Generations (1994)
GoldenEye (1995)
Shawshank Redemption, The (1994)
Stargate (1994)
Pulp Fiction (1994)
Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
Jerky Boys, The (1995)
Braveheart (1995)
Rock, The (1996)


In [16]:
movies_of_user_hates = ratings_DF[(ratings_DF['userId'] == test_subject) & (ratings_DF['rating'] < 3.0)][['movieId','rating']]
movies_of_user_hates.sort_values(by = ['rating'],ascending = False,inplace = True)
movies_of_user_hates  = movies_of_user_hates['movieId'].tolist()

print('Movies user hates.....\n')
for movie in movies_of_user_hates:
    print(movie_id_name_dict[movie])

Movies user hates.....

Grumpier Old Men (1995)
Snow White and the Seven Dwarfs (1937)
Lord of Illusions (1995)
Dolores Claiborne (1995)
Schindler's List (1993)
Philadelphia (1993)
Miracle on 34th Street (1994)
Mortal Kombat (1995)
Naked Gun 33 1/3: The Final Insult (1994)
Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994)
Operation Dumbo Drop (1995)
Nightmare Before Christmas, The (1993)
Super Mario Bros. (1993)
Beverly Hills Cop III (1994)
Richie Rich (1994)
Tank Girl (1995)
While You Were Sleeping (1995)
Poison Ivy II (1996)
Nell (1994)
Little Women (1994)
Heavenly Creatures (1994)
Prophecy, The (1995)
Mighty Morphin Power Rangers: The Movie (1995)
Casper (1995)
Postman, The (Postino, Il) (1994)
Oliver & Company (1988)


In [21]:
# For making Top N Recommendations
# --------------------------------
full_train_set = data.build_full_trainset()
anti_test_set_for_user  = anti_test_set(test_subject, full_train_set)

algo.fit(full_train_set)
predictions = algo.test(anti_test_set_for_user)

recommendations = []
print ("\nWe recommend:\n")
for user_id, movie_id, avg_rating, estimated_rating, _ in predictions:
    recommendations.append((int(movie_id), estimated_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(movies_DF[movies_DF['movieId']==ratings[0]]['title'].values[0])


We recommend:

Fight Club (1999)
Dark Knight, The (2008)
American Beauty (1999)
Matrix, The (1999)
Star Wars: Episode VI - Return of the Jedi (1983)
Sin City (2005)
Treasure of the Sierra Madre, The (1948)
Donnie Darko (2001)
Shrek (2001)
Third Man, The (1949)


In [25]:
# For measuring accuracy
# -----------------------
train_set, test_set = train_test_split(data, test_size=.25, random_state=1)
algo.fit(train_set)
accuracy_predictions = algo.test(test_set)
print('RMSE: ',RecommenderMetrics.RMSE(accuracy_predictions))
print('MAE: ',RecommenderMetrics.MAE(accuracy_predictions))

RMSE:  0.9023674654734177
MAE:  0.6977606245605574


In [26]:
# For measuring Hit Rate, Cumulative Hit Rate, Rating Hit Rate, Average Reciprocal Hit Rate
# ------------------------------------------------------------------------------------------
LOOCV = LeaveOneOut(n_splits=1, random_state=1)
for train, test in LOOCV.split(data):
    LOOCV_train = train
    LOOCV_test = test
    #print("TRAIN:", train, "TEST:", test)
algo.fit(LOOCV_train)
leave_one_out_predictions = algo.test(LOOCV_test)

# Anti-test set: The ratings are all the ratings that are not in the trainset, 
#i.e. all the ratings rui where the user u is known, 
#the item i is known, but the rating rui is not in the trainset.
LOOCV_anti_test_set = LOOCV_train.build_anti_testset()

# Build predictions for all ratings not in the training set
all_predictions_loocv = algo.test(LOOCV_anti_test_set)

n = 10

# Get top N Recommendations for each user to calculate HR, RHR,ARHR
top_n_predicted = RecommenderMetrics.GetTopN(all_predictions_loocv, n)
print(RecommenderMetrics.HitRate(top_n_predicted, leave_one_out_predictions))
print(RecommenderMetrics.CumulativeHitRate(top_n_predicted, leave_one_out_predictions))
print(RecommenderMetrics.AverageReciprocalHitRank(top_n_predicted, leave_one_out_predictions))

0.028315946348733235
0.028315946348733235
0.013000023655761361


In [29]:
# For measuring Coverage, Diversity and Novelty
# ----------------------------------------------

n = 10
full_train_set = data.build_full_trainset()
full_anti_test_set = full_train_set.build_anti_testset()
algo.fit(full_train_set)
all_predictions_full = algo.test(full_anti_test_set)
top_n_predicted = RecommenderMetrics.GetTopN(all_predictions_full, n)

# Coverage
print('Computing Coverage')
print("Coverage: ", RecommenderMetrics.UserCoverage(top_n_predicted , full_train_set.n_users, ratingThreshold=4.0))

# Diversity
# Get similarities
# Compute similarty matrix between items so we can measure diversity
sim_options = {'name': 'cosine', 'user_based': False}
sims_algo = KNNBaseline(sim_options=sim_options)
sims_algo.fit(full_train_set)
print('Computing Diversity')
print('Diversity: ',RecommenderMetrics.Diversity(top_n_predicted, sims_algo))

# Novelty
# Get popularity ranking
print('Novelty:',RecommenderMetrics.Novelty(top_n_predicted,get_popularity_rank()))

Computing Coverage
Coverage:  0.9463487332339792
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing Diversity
Computing the cosine similarity matrix...
Done computing similarity matrix.
Diversity:  0.048406542803952735
Novelty: 489.6081036745407
